from sklearn.model_selection import GridSearchCV

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

## Set grid parameters
###### logistic regression params
    lr_param_grid = {'C': [.001,.001,.01,.1,1,10], 'penalty': [l1,l2]}
###### svc params
    svc_param_grid = {'C': [1,10,100,1000], 'gamma': [.0001,.001,.01], 'kernel': ['linear', 'rbf', 'sigmoid']}
###### decision tree params
    dt_param_grid = {'max_depth': [1,5,10], 'min_samples_split': [.1,1 ,10], 'min_samples_leaf': [.1,.5,5], 'max_features': [2,6,10]}
###### xgb params
    xgb_param_grid = {"learning_rate": [.01,0.1,1],'max_depth': [2,6,10],'min_child_weight': [5,10,15],'n_estimators': [50, 100, 250]}
###### adaboost params
    ab_param_grid = {"learning_rate": [.01,0.1,1],'algorithm': ['SAMME', 'SAMME.R'],'n_estimators': [50, 100, 250]}
###### gradient boost params
    gb_param_grid = {'learning_rate': [.01,0.1,1],'max_depth': [2,6,10],'min_samples_leaf': [1,3,6],'min_samples_split': [1,2,4],'n_estimators': [50, 100, 250]}
###### random forest params
    rf_param_grid = {'max_depth': [2,6,10],'min_samples_leaf': [1,3,6],'min_samples_split': [1,2,4],'n_estimators': [5, 10, 25]}

## GridSearch parameters
    GridSearchCV(model, param_grid, scoring=None, n_jobs=-1, refit=True, cv=3, verbose=0, pre_dispatch=‘2*n_jobs’, return_train_score=False)

## Hyperparamter Visualizations for Classification

#### Max Tree Depth
    max_depths = np.linspace(1, 32, 32, endpoint=True)
    train_results = []
    test_results = []
    for max_depth in max_depths:
       dt = DecisionTreeClassifier(criterion='entropy', max_depth=max_depth)
       dt.fit(x_train, y_train)
       train_pred = dt.predict(x_train)
       false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
       roc_auc = auc(false_positive_rate, true_positive_rate)
       ###### Add auc score to previous train results
       train_results.append(roc_auc)
       y_pred = dt.predict(x_test)
       false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
       roc_auc = auc(false_positive_rate, true_positive_rate)
       ###### Add auc score to previous test results
       test_results.append(roc_auc)
    plt.figure(figsize=(12,6))
    plt.plot(max_depths, train_results, 'b', label='Train AUC')
    plt.plot(max_depths, test_results, 'r', label='Test AUC')
    plt.ylabel('AUC score')
    plt.xlabel('Tree depth')
    plt.legend()
    plt.show()
#### Minimum Sample Split
    min_samples_splits = np.linspace(0.1, 1.0, 10, endpoint=True)
    train_results = []
    test_results = []
    for min_samples_split in min_samples_splits:
       dt = DecisionTreeClassifier(criterion='entropy', min_samples_split=min_samples_split)
       dt.fit(x_train, y_train)
       train_pred = dt.predict(x_train)
       false_positive_rate, true_positive_rate, thresholds =    roc_curve(y_train, train_pred)
       roc_auc = auc(false_positive_rate, true_positive_rate)
       train_results.append(roc_auc)
       y_pred = dt.predict(x_test)
       false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
       roc_auc = auc(false_positive_rate, true_positive_rate)
       test_results.append(roc_auc)
    plt.figure(figsize=(12,6))
    plt.plot(min_samples_splits, train_results, 'b', label='Train AUC')
    plt.plot(min_samples_splits, test_results, 'r', label='Test AUC')
    plt.xlabel('Min. Sample splits')
    plt.legend()
    plt.show()
#### Minimum Sample Leaf
    min_samples_leafs = np.linspace(0.1, 0.5, 5, endpoint=True)
    train_results = []
    test_results = []
    for min_samples_leaf in min_samples_leafs:
       dt = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=min_samples_leaf)
       dt.fit(x_train, y_train)
       train_pred = dt.predict(x_train)
       false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
       roc_auc = auc(false_positive_rate, true_positive_rate)
       train_results.append(roc_auc)
       y_pred = dt.predict(x_test)
       false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
       roc_auc = auc(false_positive_rate, true_positive_rate)
       test_results.append(roc_auc)
    plt.figure(figsize=(12,6))    
    plt.plot(min_samples_leafs, train_results, 'b', label='Train AUC')
    plt.plot(min_samples_leafs, test_results, 'r', label='Test AUC')
    plt.ylabel('AUC score')
    plt.xlabel('Min. Sample Leafs')
    plt.legend()
    plt.show()
#### Maximum Features
    max_features = list(range(1,x_train.shape[1]))
    train_results = []
    test_results = []
    for max_feature in max_features:
       dt = DecisionTreeClassifier(criterion='entropy', max_features=max_feature)
       dt.fit(x_train, y_train)
       train_pred = dt.predict(x_train)
       false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
       roc_auc = auc(false_positive_rate, true_positive_rate)
       train_results.append(roc_auc)
       y_pred = dt.predict(x_test)
       false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
       roc_auc = auc(false_positive_rate, true_positive_rate)
       test_results.append(roc_auc)
    plt.figure(figsize=(12,6))
    plt.plot(max_features, train_results, 'b', label='Train AUC')
    plt.plot(max_features, test_results, 'r', label='Test AUC')
    plt.ylabel('AUC score')
    plt.xlabel('max features')
    plt.legend()
    plt.show()
#### Tree Depth (Regressor using R2 and MSE)
    max_depths = np.linspace(1, 30, 30, endpoint=True)
    mse_results = []
    r2_results = []
    for max_depth in max_depths:
        regressor = DecisionTreeRegressor(max_depth=max_depth, random_state=45)
        regressor.fit(x_train, y_train)
        y_pred = regressor.predict(x_test)
        score = performance(y_test, y_pred)
        r2_results.append(score[0])
        mse_results.append(score[1])
    plt.figure(figsize=(12,6))
    plt.plot(max_depths, r2_results, 'b', label='R2')
    plt.xlabel('Tree Depth')
    plt.ylabel('R-squared')
    plt.legend()
    plt.show()
    plt.figure(figsize=(12,6))
    plt.plot(max_depths, mse_results, 'r', label='MSE')
    plt.xlabel('Tree Depth')
    plt.ylabel('MSE')
    plt.legend()
    plt.show()